# numpy RuntimeError handling

When we encounter numerical errors, what happens?

## Extreme values with regular Python floats

In [8]:
# Multiply some big numbers
a = 1e200
b = 1e200
print(f"{a=}")
print(f"{b=}")
c = a * b
print(f"{c=}")

# Make something really small
d = 1 / (a * b)
print(f"{d=}")

a=1e+200
b=1e+200
c=inf
d=0.0


When going over around 1e301, we get an overflow, but no error is thrown; the value of `c` is just set to `inf`. For < 1e-301, we get 0.0.

In [9]:
# Try divide by zero
d = a / 0

ZeroDivisionError: float division by zero

Dividing by zero throws an exception.

In [14]:
# NaN experiment
e = float("NaN") + 1
print(f"{e=}")

e=nan


However, NaNs are fine. 

So, only divide by zero throws an exception in standard Python. Overflows, underflows and NaNs go along without complaint, which could make finding the source of errors difficult.

## Extreme values with numpy

What about numpy?

In [51]:
import numpy as np

# Overflow
a = np.array([1e200])
b = np.array([1e200])
c = a * b
print(f"{c=}")

# Attempt underflow
d = 1 / c # doesn't throw
d = 1 / (a * b) # throws overflow
print(f"{d=}")

# Div by zero
e = a / 0
print(f"{e=}")

# Force NaN
f = np.sqrt(np.NaN)
print(f"{f=}")

c=array([inf])
d=array([0.])
e=array([inf])
f=nan


/tmp/ipykernel_116772/440360440.py:6: RuntimeWarning: overflow encountered in multiply
  c = a * b
/tmp/ipykernel_116772/440360440.py:11: RuntimeWarning: overflow encountered in multiply
  d = 1 / (a * b) # throws overflow
/tmp/ipykernel_116772/440360440.py:15: RuntimeWarning: divide by zero encountered in divide
  e = a / 0


Warns about overflows (can't seem to produce an underflow) and divide by zeros, but is fine with NaNs.

However, these are just warnings; we need to change the calculations to avoid these situations. `inf`s and `0`s propagate to cause other errors that can throw Python exceptions or ultimately result in Fortran `STOP 1`s, and halt the optimisation run. Warnings can be caught, but this is cumbersome and not Pythonic. It's possible to get numpy to error in these cases:

In [53]:
np.seterr(all="raise")

# Overflow
a = np.array([1e200])
b = np.array([1e200])
c = a * b

FloatingPointError: overflow encountered in multiply

This can then be caught:

In [55]:
np.seterr(all="raise")

# Overflow
a = np.array([1e200])
b = np.array([1e200])

try:
    c = a * b
except FloatingPointError:
    # Make very big instead: kludge
    c = 1e6

print(f"{c=}")

c=1000000.0


This approach has drawbacks, however. Firstly, this will only work if `np.seterr()` is set to raise, so that exceptions are raised for everything. This will catch these errors that cause problems later on, but it will also catch any error (even non-fatal ones) which will have to all be fixed before an optimisation run succeeds. This could create a huge number of errors that need fixing, even if the errors are innocuous. It also goes against the default numpy behaviour, which is to warn to stderr.

The second problem is that it will only catch errors with numpy arrays: regular Python float errors/extreme values won't be caught.

Perhaps a better approach is to use normal numpy error warning but assert only on values that are causing problems. Any `AssertionError` can then be handled. This gives the code a chance to keep going, whilst handling errors in key variables with asserts.

In [62]:
import logging

logger = logging.getLogger(__name__)

np.seterr(all="warn")

# Overflow
a = np.array([1e200])
b = np.array([1e200])

# Raise warning
c = a * b

try:
    assert c <= 1e6
except AssertionError:
    # Make very big instead: kludge
    c = 1e6
    logger.warning("Corrected c")

print(f"{c=}")

/tmp/ipykernel_116772/3036535714.py:12: RuntimeWarning: overflow encountered in multiply
  c = a * b
Corrected c


c=1000000.0
